# PriorCVAE on Lotka Volterra

In [1]:
import random as rnd

import numpy as np 
import matplotlib.pyplot as plt
import numpyro
from numpyro.infer import Predictive
from numpyro.diagnostics import hpdi
numpyro.set_host_device_count(4)
import jax
import optax
from jax import random
import jax.numpy as jnp

from priorCVAE_copy.models import MLPEncoder, MLPDecoder, VAE
from priorCVAE_copy.priors import GP, SquaredExponential
from priorCVAE_copy.mcmc import run_mcmc_vae, vae_mcmc_inference_model
from priorCVAE_copy.datasets import GPDataset, LVDataset
from priorCVAE_copy.trainer import VAETrainer
from priorCVAE_copy.losses import SquaredSumAndKL

import jax.config as config
config.update("jax_enable_x64", True)

In [11]:
import pints
import pints_jax
import pints.toy as toy
import pints.toy.stochastic
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

In [3]:
args = {
        "conditional": True,
        
        # architecture
        "input_dim" : 80,
        "hidden_dim": 60,
        "latent_dim": 40,
        
        # VAE training    
        "batch_size": 1,
        "num_iterations": 3000,
        "learning_rate": 1e-3,
        "vae_var": 1.,

        # MCMC inference
        "true_ls": 0.2,
        "num_warmup": 1000,
        "num_mcmc_samples": 4000,
        "num_chains": 4,
        "thinning": 1,
       }

## Generate Data
**Problem:** how can we get a general enough data set, including different initial population densities and parameters?   
Parameters are easier to generate, uniform priors will be fine. What about initial population densities? What are common initial circumstances for the Lotka Volterra model? What are considered as general?

In [13]:
model = toy.LotkaVolterraModel()

# Example of a toy data with suggested parameter values
times = np.linspace(0, 20,30)
params = np.array([0.5,0.15,1,.3])
values = model.simulate(params, times)
print(values)

[[3.40119738 1.38629436]
 [4.14076041 1.51517924]
 [4.9048173  1.94022877]
 [5.43125688 2.85746673]
 [5.29026188 4.41232682]
 [4.32622326 6.05320352]
 [3.13973391 6.55419567]
 [2.32769919 5.74252599]
 [1.93807723 4.45222991]
 [1.83768459 3.28823187]
 [1.9353099  2.4303788 ]
 [2.19291194 1.8641874 ]
 [2.60174615 1.531973  ]
 [3.16245491 1.39180089]
 [3.8628571  1.44136876]
 [4.63781229 1.74218394]
 [5.29390505 2.45394217]
 [5.44322374 3.78701173]
 [4.74482306 5.53100522]
 [3.54363405 6.55208826]
 [2.56668017 6.1382553 ]
 [2.03872976 4.92456555]
 [1.84772342 3.67661891]
 [1.88042557 2.70413433]
 [2.08227555 2.03881964]
 [2.43646554 1.62835841]
 [2.94241744 1.42151795]
 [3.59586826 1.39942135]
 [4.35741927 1.59683799]
 [5.09091762 2.13470385]]


In [2]:
generator = LVDataset()
a,b,c = generator.simulatedata(3)

Traced<ShapedArray(float64[30,2])>with<DynamicJaxprTrace(level=1/0)>


In [3]:
np.shape(c)

(3, 30, 2)

In [4]:
print(b)

[[0.83107835 1.25096046 0.88570101 1.52500643]
 [0.14492511 0.04063634 0.95985286 0.17269169]
 [0.59302312 0.46028562 0.03866878 1.86871807]]


In [7]:
np.shape(a)

(30, 30)

In [9]:
a = np.array([1,2,3,4])
a.pop(0)

AttributeError: 'numpy.ndarray' object has no attribute 'pop'